# Task2
Let's take users registered in [2019-01-15; 2019-01-31]. Build a prediction of how many messages they generate in total on the 15th day of life.

```
with reg as(
    select *
    from 'regular_users'
    where true
        and (date_registration between '2018-12-05' and '2019-01-14')
),

msg as(
    select *
    from 'messages'
    where true
        and (date_created between '2018-12-05' and '2019-01-30')
),

choosing_15day_lt as(
    select
        date_diff(msg.date_created, ru.date_registration, day) as num_day,
        sum(n_messages) as n_messages,
        count(distinct id_ru) as dist_ru,
        count(id_ru) as ru
    from reg
        left join msg using(id_ru)
    where true
        and date_diff(msg.date_created, ru.date_registration, day) < 15
    group by 1
),

msg_per_user as(
    select
        num_day,
        sum(n_messages) over(order by num_day) / (select count(id_ru) from reg) as n_messages
    from choosing_15day_lt
),

curve as(
    select
        num_day as lt,
        n_messages / max(n_messages) over() as perc
    from msg_per_user
    order by 1
),

reg_to_predict as(
    select *
    from 'regular_users'
    where true
    and (date_registration between '2019-01-15' and '2019-01-30')
),

count_msg as(
    select
        date_diff('2019-01-30', date_registration, day) as lt,
        sum(n_messages) as n_messages
    from reg_to_predict
        join msg using(id_ru)
    where true
    and date_diff(mes.date_created, dec_ru.date_registration, day) <= 15
    group by 1
),

more_than_15 as(
    select
        if(lt > 15, 15, lt) as lt,
        n_messages
    from count_msg
)

select
    sum(n_messages / perc)
from more_than_15
join curve using(lt)
```